In [8]:
def format_list(lst: list[str]) -> str:return "'" + "', '".join(lst) + "'"

list1 = ["one", "two", "three"]

formatted_list = format_list(list1)
print(formatted_list)

'one', 'two', 'three'


In [9]:
import os
from openai import OpenAI
from pinecone import Pinecone

from src.embedder import Embedder
from src.db import PineconeDB

# get the keys and create clients
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
LANGCHAIN_API_KEY = os.environ["LANGCHAIN_API_KEY"]   # TODO: where is this actually used???

openai_client = OpenAI(api_key=OPENAI_API_KEY)
pinecone_client = Pinecone(api_key=PINECONE_API_KEY)

In [10]:
# index_name = "docu-help"
# namespace = "crawlee"

In [13]:
embedder = Embedder(openai_client)
db = PineconeDB(pinecone_client)
print(db)

PineconeDB(indexes=[])


In [14]:
test_index = "testing123"
test_index2 = "testing456"

In [15]:
db.create_index(test_index, embedder)
db.create_index(test_index2, embedder)

Creating index testing123...
Creating index testing456...


In [16]:
print(db)

PineconeDB(indexes=[testing123 (namespaces: 0), testing456 (namespaces: 0)])


In [17]:
db.delete_index(test_index)
db.delete_index(test_index2)

Deleting index testing123...
Deleting index testing456...


In [18]:
print(db)

PineconeDB(indexes=[])


In [11]:
import torch
cov_var = torch.full(size=(3, ), fill_value=0.5)


In [12]:
torch.diag(cov_var)

tensor([[0.5000, 0.0000, 0.0000],
        [0.0000, 0.5000, 0.0000],
        [0.0000, 0.0000, 0.5000]])

In [14]:
import torch
import numpy as np
from torch.distributions import MultivariateNormal

class Actor(torch.nn.Module):
    def __init__(self, obs_dim, act_dim):
        super().__init__()
        self.fc1 = torch.nn.Linear(obs_dim, 64)
        self.fc2 = torch.nn.Linear(64, 64)
        self.fc3 = torch.nn.Linear(64, act_dim)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class Agent:
    def __init__(self, obs_dim, act_dim):
        self.actor = Actor(obs_dim, act_dim)
        self.cov_mat = torch.eye(act_dim)
        
    def get_action(self, obs):
        obs = torch.tensor(obs, dtype=torch.float32)
        mean = self.actor(obs)
        dist = MultivariateNormal(mean, self.cov_mat)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        return action.detach().numpy(), log_prob.detach()

# Example usage
obs_dim = 4
act_dim = 2
agent = Agent(obs_dim, act_dim)

# Generate a random observation
obs = np.random.randn(obs_dim)

# Get the action and log probability
action, log_prob = agent.get_action(obs)

print("Observation:", obs)
print("Action:", action)
print("Log Probability:", log_prob)

Observation: [-0.04799651 -0.32498188 -0.6190602  -0.29137785]
Action: [-0.2177402   0.80992997]
Log Probability: tensor(-2.2678)


In [15]:
import torch.distributions as dis
p = dis.Normal(loc=0, scale=1)
q = dis.Normal(loc=0.1, scale=1)
x = q.sample(sample_shape=(10_000_000,))
truekl = dis.kl_divergence(p, q)
print("true", truekl)
logr = p.log_prob(x) - q.log_prob(x)
k1 = -logr
k2 = logr ** 2 / 2
k3 = (logr.exp() - 1) - logr
for k in (k1, k2, k3):
    print((k.mean() - truekl) / truekl, k.std() / truekl)

true tensor(0.0050)
tensor(-0.0047) tensor(20.0085)
tensor(0.0033) tensor(1.4185)
tensor(0.0008) tensor(1.4174)
